# Migration of old table structure to the new one


### Table columns:

In [14]:
import requests
import numpy as np
from itertools import product
import pandas as pd
import json

In [22]:
payload = {'q': 'select * from water_risk_indicators limit 2'}
r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(0)

Empty DataFrame
Columns: [basinid, cartodb_id, indicator, label, model, period, period_value, the_geom, the_geom_webmercator, type, value, year]
Index: []

### Template sql that we will use to generate the migration query

Sql example for projected data:
```sql
SELECT  basinid, 'water supply' as indicator, bt2024tl as label,'optimistic' as model, 'year' as period, null as period_value, 'absolute' as type, bt2024tl as value, 2020 as year  FROM aqueduct_projections_20150309
```

Sql template for projected data:
```sql
SELECT  basinid, {{indicator}} as indicator, {II}{YY}{SS}{R}l as label, {{scenario}} as model, {{period}} as period, {{period_value}} as period_value, {{data_type}} as type, {II}{YY}{SS}{R}r as value, {{year}} as year  FROM aqueduct_projections_20150309
```

Sql example for projected data:
```sql
SELECT  basinid, 'water supply' as indicator, bws_cat  as label,'historic' as model, 'year' as period, null as period_value, 'absolute' as type, bws as value, 2014 as year  FROM aqueduct_global_dl_20150409
```
Sql template for current conditions data:

```sql
SELECT  basinid, {{indicator}} as indicator, {II}_cat  as label,'historic' as model, 'year' as period, null as period_value, 'absolute' as type, {II} as value, 2014 as year  FROM aqueduct_global_dl_20150409
```


{{indicator}}  
{{scenario}}  
{{period_type}}  
{{period_value}}  
{{data_type}}  
{{year}}  
 
For future projections: 
 
{II}{YY}{SS}{T}{X}  
{II}	indicator code  
{YY}	year code  
{SS}	scenario code  
{T}	data type code  
{X}	suffix  

{{indicator}} / {II}	Indicator codes

	water stress (ws)
	seasonal variability (sv)
	water demand (ut)
	water supply (bt)

{{period}} will be one of this: 

    month, 
    year, 
    quarter
    
{{period_value}} will depend on period_type value:

    month: 1:12 
    year: null
    quarter: 1:4
    
{{scenario}} / {SS}	Scenario codes:

    historic ( bs ) 
    pessimistic ( 38 ) ssp3 rcp85
    optimistic ( 24 ) ssp2 rcp45
    bau ( 28 ) ssp2 rcp85

{{data_type}} / {T}	Data types:

    absolute ( t )
    change from baseline ( c )
    uncertainity ( u )
    
{{year}} / {YY}	Year codes:

    2014 ( 00 )
	2020 ( 20 )
	2030 ( 30 )
	2040 ( 40 )
	
	
{X}	Suffixes

    l	label string
    r	raw value

In [5]:
## 'SELECT  basinid, {{indicator}} as indicator, {II}{YY}{SS}{R}l as label, {{scenario}} as model, {{period_type}} as period, {{period_value}} as period_value, {{data_type}} as type, {II}{YY}{SS}{R}r as value, {{year}} as year  FROM aqueduct_projections_20150309'
sqlTemplate =["SELECT  basinid, ",' as indicator, ', 'l::text as label, ', ' as model, ', ' as period, ', ' as period_value, ',' as type, ','r as value, ', ' as year  FROM aqueduct_projections_20150309']
freplacedict={
    'indicator':{
        '{{indicator}}': ['\'water_stress\'', '\'seasonal_variability\'', '\'water_demand\'', '\'water_supply\''],
        '{II}': ['ws', 'sv', 'ut', 'bt']
                },
    'scenario':{
        '{{scenario}}': ['\'historic\'','\'pessimistic\'','\'optimistic\'','\'bau\''],
        '{SS}': ['00','38','24','28']
    },
    'period_type':{
        '{{period_type}}': [ '\'year\'','\'month\'', '\'quarter\'']
    },
    'period_value':{
        '{{period_value}}': {
            'month':{
                'min': 1,
                'max': 12
                }, 
            'year': 'NULL', 
            'quarter':{
                'min': 1,
                'max': 4
                }
        }
    },
    'data_type':{
        '{{data_type}}': ['\'absolute\'','\'change_from_baseline\'','\'uncertainity\''],
        '{T}': ['t','c','u']
    },
    'year':{
        '{{year}}': [2014, 2020, 2030, 2040],
        '{YY}': ['bs','20','30','40']
    },  
};


In [6]:
d = ['\'Baseline Water Stress\'',
'\'Interannual Variability\'',
'\'Seasonal Variability\'',
'\'Flood Occurrence\'',
'\'Drought Severity\'',
'\'Upstream Storage\'',
'\'Groundwater Stress\'',
'\'Return Flow Ratio\'',
'\'Upstream Protected Land\'',
'\'Media Coverage\'',
'\'Access to Water\'',
'\'Threatened Amphibians\'']
r=[]
for l in d:
    r.append(l.lower().strip(' \t\n\r').replace(' ', '_'))
print(r)

["'baseline_water_stress'", "'interannual_variability'", "'seasonal_variability'", "'flood_occurrence'", "'drought_severity'", "'upstream_storage'", "'groundwater_stress'", "'return_flow_ratio'", "'upstream_protected_land'", "'media_coverage'", "'access_to_water'", "'threatened_amphibians'"]


In [22]:
#SELECT  basinid, {{indicator}} as indicator, {II}_cat  as label,'historic' as model, 'year' as period, null as period_value, 'absolute' as type, {II} as value, 2014 as year  FROM aqueduct_global_dl_20150409

current_sqlTemplate=['SELECT  basinid, ',' as indicator, ', '_cat  as label,\'historic\' as model, \'year\' as period, null as period_value, \'absolute\' as type, ', ' as value, 2014 as year  FROM aqueduct_global_dl_20150409', '']
currentplacedict={
    'indicator':{
        '{{indicator}}': ["'baseline_water_stress'", "'interannual_variability'", "'seasonal_variability'", "'flood_occurrence'", "'drought_severity'", "'upstream_storage'", "'groundwater_stress'", "'return_flow_ratio'", "'upstream_protected_land'", "'media_coverage'", "'access_to_water'", "'threatened_amphibians'"],
        '{II}': ['bws','wsv','sv','hfo','dro','stor','gw','wri','eco_s','mc','wcg','eco_v']
        
                },
    'scenario':{
        '{{scenario}}': ['\'historic\''],
    },
    'period_type':{
        '{{period_type}}': [ '\'year\'','\'month\'', '\'quarter\'']
    },
    'period_value':{
        '{{period_value}}': {
            'month':{
                'min': 1,
                'max': 12
                }, 
            'year': 'NULL', 
            'quarter':{
                'min': 1,
                'max': 4
                }
        }
    },
    'data_type':{
        '{{data_type}}': ['\'absolute\'','\'scores\''],
        '{T}': ['','s']
    },
    'year':{
        '{{year}}': [2014]
    },  
};

In [23]:
#{{indicator}},{II},{II} 

def iter_items_c(dictionary):
    replaceList=[]
    for x in range(len(dictionary['indicator']['{II}'])):
        if (dictionary['indicator']['{II}'][x]!='bws'):
            replaceList.append([dictionary['indicator']['{{indicator}}'][x],
                           dictionary['indicator']['{II}'][x],
                           dictionary['indicator']['{II}'][x],
                          ''
                          ]) 
                
    return replaceList
       

In [9]:
## 'SELECT  basinid, {{indicator}} as indicator, {II}{YY}{SS}{R}l as label, {{scenario}} as model, {{period_type}} as period, {{period_value}} as period_value, {{data_type}} as type, {II}{YY}{SS}{R}r as value, {{year}} as year  FROM aqueduct_projections_20150309'

def iter_items(dictionary):
    replaceList=[]
    for t in range(3):
        for x, y, s in product(range(4), repeat=3):
            if ( (dictionary['year']['{YY}'][y]!='bs' and dictionary['scenario']['{SS}'][s]!='00') or (dictionary['year']['{YY}'][y] =='bs' and dictionary['scenario']['{SS}'][s] =='00' and (dictionary['indicator']['{II}'][x]!='sv' and dictionary['indicator']['{II}'][x]!='bt' and dictionary['indicator']['{II}'][x]!='ut' and dictionary['data_type']['{T}'][t]!='c' and dictionary['data_type']['{T}'][t]!='u'))):
                column = dictionary['indicator']['{II}'][x] + dictionary['year']['{YY}'][y] + dictionary['scenario']['{SS}'][s] + dictionary['data_type']['{T}'][t]
                replaceList.append([dictionary['indicator']['{{indicator}}'][x],
                           column,
                           dictionary['scenario']['{{scenario}}'][s],
                           dictionary['period_type']['{{period_type}}'][0],
                           dictionary['period_value']['{{period_value}}']['year'],
                           dictionary['data_type']['{{data_type}}'][t],
                           column,
                           dictionary['year']['{{year}}'][y],
                          ''
                          ]) 
                
    return replaceList

In [10]:
def sql_generator(sql_sentence, replacement):
    return ''.join([str(a) + str(b) for a,b in zip(sql_sentence,replacement)])

In [11]:
import sys
sys.modules['itertools']

<module 'itertools' (built-in)>

In [25]:
items = iter_items(freplacedict)
sql_sentences = []
for item in items:
    sql_sentences.append(sql_generator(sqlTemplate, item))

In [24]:
citems = iter_items_c(currentplacedict)
sql_sentences = []
for item in citems:
    sql_sentences.append(sql_generator(current_sqlTemplate, item))
    
' UNION ALL '.join(sql_sentences)
    
        

"SELECT  basinid, 'interannual_variability' as indicator, wsv_cat  as label,'historic' as model, 'year' as period, null as period_value, 'absolute' as type, wsv as value, 2014 as year  FROM aqueduct_global_dl_20150409 UNION ALL SELECT  basinid, 'seasonal_variability' as indicator, sv_cat  as label,'historic' as model, 'year' as period, null as period_value, 'absolute' as type, sv as value, 2014 as year  FROM aqueduct_global_dl_20150409 UNION ALL SELECT  basinid, 'flood_occurrence' as indicator, hfo_cat  as label,'historic' as model, 'year' as period, null as period_value, 'absolute' as type, hfo as value, 2014 as year  FROM aqueduct_global_dl_20150409 UNION ALL SELECT  basinid, 'drought_severity' as indicator, dro_cat  as label,'historic' as model, 'year' as period, null as period_value, 'absolute' as type, dro as value, 2014 as year  FROM aqueduct_global_dl_20150409 UNION ALL SELECT  basinid, 'upstream_storage' as indicator, stor_cat  as label,'historic' as model, 'year' as period, nu

gu
shape_leng
shape_area
basinid
country
basin_name
withdrawal
consumptio
ba
bws
bws_s
bws_cat
wsv
wsv_s
wsv_cat
sv
sv_s
sv_cat
hfo
hfo_s
hfo_cat
dro
dro_s
dro_cat
bt
stor
stor_s
stor_cat
gw
gw_s
gw_cat
wri
wri_s
wri_cat
eco_s
eco_s_s
eco_s_cat
mc
mc_s
mc_cat
eco_v
eco_v_s
eco_v_cat
wcg
wcg_s
wcg_cat
def_pqual
def_regrep
w_semico
_default
w_constr
w_chem
w_power
w_mine
w_oilgas
def_pquant
w_agr
w_foodbv
w_tex
owr_cat

|  to mantain  | to Change  | 
|---|---|
|5. Extremely high (>1.0)    |  Extremely high (>80%) |
|4. High (0.75-1.0)          |  High (40-80%) |
|3. Medium to high (0.5-0.75)|  Medium-high (20-40%) |
|2. Low to medium (0.25-0.5) |  Low-medium (10-20%) |
|1. Low (<0.25)              |  Low (<10%) |
|Arid and low water use |  | 
|No data |  |